In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [4]:
from typing import Optional, List, Dict, Tuple

In [5]:
from core.models.conv_vqvae import ConvVQMotionModel
from configs.config import cfg, get_cfg_defaults
from core.models.conformer_vqvae import ConformerVQMotionModel

In [6]:
path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml"
cfg = get_cfg_defaults()
print("loading config from:", path)
cfg.merge_from_file(path)
cfg.freeze()

loading config from: /srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml


In [7]:
vqvae_model = ConformerVQMotionModel(
    cfg.vqvae,
    # is_distributed=self.is_distributed,
)

Sync is turned on False


In [21]:
total = sum(p.numel() for p in vqvae_model.motionEncoder.parameters() if p.requires_grad)
print("Total training params: %.2fM" % (total / 1e6))


Total training params: 16.58M


In [10]:
x = torch.randn((1,80,271))

In [13]:
y, ind, lss = vqvae_model(x)

In [14]:
y.shape

torch.Size([1, 80, 271])

In [22]:
cfg.vqvae.depth

6

## Calc mean comman

In [7]:
hml = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D_SMPL"
aist = "/srv/hays-lab/scratch/sanisetty3/music_motion/AIST_SMPL"
cm ="/srv/hays-lab/scratch/sanisetty3/music_motion/Choreomaster_SMPL"

In [11]:
mh = np.load(os.path.join(hml, "Mean.npy"))
ma = np.load(os.path.join(aist, "Mean.npy"))
mc = np.load(os.path.join(cm, "Mean.npy"))

sh = np.load(os.path.join(hml, "Std.npy"))
sa = np.load(os.path.join(aist, "Std.npy"))
sc = np.load(os.path.join(cm, "Std.npy"))


In [13]:
mh.shape

(271,)

In [17]:
m = np.mean([mh, ma,mc] , 0)
s = np.mean([sh,sa,sc] , 0)

In [20]:
np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/pretrained/common/Std.npy" , s)

## Muse stuff

In [31]:
from functools import partial

In [27]:
from dataclasses import dataclass

@dataclass
class AttentionParams:
    dim: int = 768
    dim_head: int = 96
    heads: int = 8
    causal: bool = False
    qk_norm: bool = False
    qk_norm_scale: int = 8
    dropout: float = 0.0
    cross_attn_tokens_dropout: float = (0.0,)
    add_null_kv: bool = False
    flash: bool = False


In [28]:
a = AttentionParams()

In [33]:
a.causal = True

In [38]:
from enum import Enum

class PositionalEmbeddingType(Enum):
    REL= "RelativePositionBias"
    SINE= "ScaledSinusoidalEmbedding"
    ALIBI= "AlibiPositionalBias"
    ABS= "AbsolutePositionalEmbedding"
    SHAW= "ShawRelativePositionalEmbedding"


In [40]:
a = PositionalEmbeddingType.REL

In [68]:
a = torch.randn((4,100,768))

In [48]:
norm = nn.InstanceNorm1d(768)

In [51]:
def uniform(shape, min=0, max=1, device=None):
    return torch.zeros(shape, device=device).float().uniform_(0, 1)


def prob_mask_like(shape, prob, device=None):
    if prob == 1:
        return torch.ones(shape, device=device, dtype=torch.bool)
    elif prob == 0:
        return torch.zeros(shape, device=device, dtype=torch.bool)
    else:
        return uniform(shape, device=device) < prob


In [53]:
b,n,d = a.shape

In [60]:
mask_ = prob_mask_like((b, 1), 1.0 - 0.5, "cpu")


In [61]:
mask_

tensor([[False],
        [ True],
        [False],
        [False]])

In [69]:
context_mask = (a != 1026).any(dim=-1)

In [71]:
mask_

tensor([[False],
        [ True],
        [False],
        [False]])

In [63]:
context_mask = context_mask & mask_